# 1. Import Libraries

In [1]:
import os
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read(r'../config.ini')

validPath = False

datasetPaths = config.get('DataCleansing', 'DatasetPath').split(',')
for path in datasetPaths:
    try:
        print(f'Trying {path.strip()}...')
        os.chdir(path.strip())
        validPath = True
        print('Path found')
    except FileNotFoundError:
        print(f'Path "{path.strip()}" not found')

if not validPath:
    raise RuntimeError('Could not find any valid dataset path')

Trying D:\SP\Year 2 Sem 2\ST1508 Practical AI\PAICA1\datasets...
Path found


# 2. Data Cleansing (Customer)

In [2]:
ogCustomer = pd.read_csv("training-data-customer-features_v1.0.csv")
ogCustomer.head()

,country_code,customer_id,mobile_verified,num_orders_last_50days,num_cancelled_orders_last_50days,num_refund_orders_last_50days,total_payment_last_50days,num_associated_customers,first_order_datetime
0,PH,phjr7fpu,True,0,0,0,0.000000,1,2023-02-06 14:31:37
1,PH,phawdv1w,True,0,0,0,1.082516,1,2023-02-07 10:03:22
2,PH,ph9ocrr7,True,0,0,0,0.050195,2,2022-03-08 06:16:09
3,PH,phf8rsec,True,0,0,0,0.000000,3,2021-08-07 13:54:45
4,PH,ph74rzxh,True,0,0,0,0.000000,1,2022-01-28 12:57:22


In [3]:
ogCustomer.shape

(2195916, 9)

## A. Check for Missing Values

In [4]:
ogCustomer.isnull().sum()

country_code                           0
customer_id                            0
mobile_verified                        0
num_orders_last_50days                 0
num_cancelled_orders_last_50days       0
num_refund_orders_last_50days          0
total_payment_last_50days              0
num_associated_customers               0
first_order_datetime                1240
dtype: int64

In [5]:
percentageOfNa = round(ogCustomer.iloc[:,-1].isnull().sum() / len(ogCustomer) * 100, 2)
print(f'{percentageOfNa}% of the rows contain missing values. Since {percentageOfNa}% is very insignificant, we will drop the missing values')

0.06% of the rows contain missing values. Since 0.06% is very insignificant, we will drop the missing values


In [6]:
dropnaCustomer = ogCustomer.dropna()
dropnaCustomer.isnull().sum()

country_code                        0
customer_id                         0
mobile_verified                     0
num_orders_last_50days              0
num_cancelled_orders_last_50days    0
num_refund_orders_last_50days       0
total_payment_last_50days           0
num_associated_customers            0
first_order_datetime                0
dtype: int64

## B. Check for Duplicated Rows

Since our unique identifier for each row is a composite key comprised of `country_code` + `customer_id`, we will treat a row as a duplicate if its composite key is duplicated, even if the other columns are different

In [7]:
print(dropnaCustomer.duplicated(subset=['country_code', 'customer_id']).sum())

907902


As newer data records are always and nearer to the end of the table, we will use the last record of that customer and discard all other records from the same customer

In [8]:
dropDuplicatesCustomer = dropnaCustomer.drop_duplicates(keep='last', subset=['country_code', 'customer_id'])
print(dropDuplicatesCustomer.duplicated(subset=['country_code', 'customer_id']).sum())

0


## C. Check Datatypes

In [9]:
dropDuplicatesCustomer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1286774 entries, 0 to 2195915
Data columns (total 9 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   country_code                      1286774 non-null  object 
 1   customer_id                       1286774 non-null  object 
 2   mobile_verified                   1286774 non-null  bool   
 3   num_orders_last_50days            1286774 non-null  int64  
 4   num_cancelled_orders_last_50days  1286774 non-null  int64  
 5   num_refund_orders_last_50days     1286774 non-null  int64  
 6   total_payment_last_50days         1286774 non-null  float64
 7   num_associated_customers          1286774 non-null  int64  
 8   first_order_datetime              1286774 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 89.6+ MB


### i. `first_order_datetime`

In [10]:
dropDuplicatesCustomer.iloc[:,-1]

0          2023-02-06 14:31:37
1          2023-02-07 10:03:22
2          2022-03-08 06:16:09
3          2021-08-07 13:54:45
4          2022-01-28 12:57:22
                  ...         
2195911    2014-11-01 06:55:11
2195912    2022-06-25 07:03:58
2195913    2018-12-21 12:04:11
2195914    2014-03-26 06:10:24
2195915    2014-12-13 15:05:26
Name: first_order_datetime, Length: 1286774, dtype: object

As we are unsure whether all of the dates are of the same format, we can check using regular expressions

In [11]:
import re

pattern = r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"

def checkDatetimeFormat(datetimes):
    return all(re.match(pattern, dt) for dt in datetimes)

print(checkDatetimeFormat(dropDuplicatesCustomer.iloc[:,-1].values))


True


After we have ensured that they are all in the same format, we can convert them to `datetime` datatype

In [12]:
dropDuplicatesCustomer['first_order_datetime'] = pd.to_datetime(dropDuplicatesCustomer['first_order_datetime'], format='%Y-%m-%d %H:%M:%S')

C:\Users\scrab\AppData\Local\Temp\ipykernel_3240\4026684440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropDuplicatesCustomer['first_order_datetime'] = pd.to_datetime(dropDuplicatesCustomer['first_order_datetime'], format='%Y-%m-%d %H:%M:%S')


In [13]:
dropDuplicatesCustomer.iloc[:,-1].unique()[:5]

<DatetimeArray>
['2023-02-06 14:31:37', '2023-02-07 10:03:22', '2022-03-08 06:16:09',
 '2021-08-07 13:54:45', '2022-01-28 12:57:22']
Length: 5, dtype: datetime64[ns]

### ii. `mobile_verified`

As SQL does not have a boolean datatype with `T` or `F` values but instead a `BIT` with `1` and `0` values, we will convert the `mobile_verified` column to `int` datatype by converting `True` to `1` and `False` to `0`

In [14]:
dropDuplicatesCustomer.iloc[:,2].unique()

array([ True, False])

In [15]:
dropDuplicatesCustomer.iloc[:,2] = dropDuplicatesCustomer.iloc[:,2].astype('int64')

In [16]:
dropDuplicatesCustomer.iloc[:,2].unique()

array([1, 0], dtype=int64)

In [17]:
dropDuplicatesCustomer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1286774 entries, 0 to 2195915
Data columns (total 9 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   country_code                      1286774 non-null  object        
 1   customer_id                       1286774 non-null  object        
 2   mobile_verified                   1286774 non-null  int64         
 3   num_orders_last_50days            1286774 non-null  int64         
 4   num_cancelled_orders_last_50days  1286774 non-null  int64         
 5   num_refund_orders_last_50days     1286774 non-null  int64         
 6   total_payment_last_50days         1286774 non-null  float64       
 7   num_associated_customers          1286774 non-null  int64         
 8   first_order_datetime              1286774 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 98.2+ MB


## D. Exporting Cleaned Dataset

In [18]:
customer = dropDuplicatesCustomer
ogCustomerLength = len(ogCustomer)
customerLength = len(customer)

print(ogCustomer.shape)
print(customer.shape)
print(f'We removed {ogCustomerLength - customerLength} rows which is {((ogCustomerLength - customerLength) / ogCustomerLength * 100):.2f}% of the total number of rows in the dataset')

(2195916, 9)
(1286774, 9)
We removed 909142 rows which is 41.40% of the total number of rows in the dataset


Although 41.4% is quite a significant amount of data cleansed from our dataset, we still have more than 1.2 million to work with for our model training which is not too little

In [19]:
customer.to_csv('final_customer.csv', index=False)

# 3. Data Cleansing (Order)

In [20]:
ogOrder = pd.read_csv("training-data-order-features_v1.0.csv")
ogOrder.head()

,country_code,order_id,collect_type,payment_method,order_value,num_items_ordered,refund_value,order_date
0,PH,cktw-hdnh,delivery,credit card,18.379008,2.0,6.07,2023/04/09
1,PH,s9mi-zzap,delivery,antfinancial gcash,5.387819,4.0,0.00,2023/04/09
2,PH,vgow-r2i6,delivery,payment on delivery,8.526517,4.0,0.00,2023/04/09
3,PH,u7pg-7rn7,delivery,antfinancial gcash,7.452310,6.0,0.00,2023/04/09
4,PH,q6dn-z0p9,delivery,credit card,9.030052,4.0,0.62,2023/04/09


In [21]:
ogOrder.shape

(2270509, 8)

## A. Check for Missing Values

In [22]:
ogOrder.isnull().sum()

country_code         0
order_id             0
collect_type         0
payment_method       0
order_value          0
num_items_ordered    3
refund_value         0
order_date           0
dtype: int64

In [23]:
dropnaOrder = ogOrder.dropna()
dropnaOrder.isnull().sum()

country_code         0
order_id             0
collect_type         0
payment_method       0
order_value          0
num_items_ordered    0
refund_value         0
order_date           0
dtype: int64

## B. Check for Duplicated Rows

In [24]:
print(dropnaOrder.duplicated().sum())

1110


In [25]:
dropDuplicatesOrder = dropnaOrder.drop_duplicates(keep='last')
print(dropDuplicatesOrder.duplicated().sum())

0


## C. Check Datatypes

In [26]:
dropDuplicatesOrder.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2269396 entries, 0 to 2270508
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   country_code       object 
 1   order_id           object 
 2   collect_type       object 
 3   payment_method     object 
 4   order_value        float64
 5   num_items_ordered  float64
 6   refund_value       float64
 7   order_date         object 
dtypes: float64(3), object(5)
memory usage: 155.8+ MB


### i. `order_date`

In [27]:
dropDuplicatesOrder.iloc[:,-1].unique()[:5]

array(['2023/04/09', '2023/04/10', '2023/04/11', '2023/04/12',
       '2023/04/13'], dtype=object)

As the `order_date` is in `object` datatype, we will convert its format from `YYYY/MM/DD` TO `YYYY-MM-DD` to ensure consistency in the format of the dates

In [28]:
dropDuplicatesOrder['order_date'] = dropDuplicatesOrder['order_date'].str.replace('/', '-', regex=False)
dropDuplicatesOrder['order_date'] = pd.to_datetime(dropDuplicatesOrder['order_date'], format='%Y-%m-%d')

C:\Users\scrab\AppData\Local\Temp\ipykernel_3240\909367782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropDuplicatesOrder['order_date'] = dropDuplicatesOrder['order_date'].str.replace('/', '-', regex=False)
C:\Users\scrab\AppData\Local\Temp\ipykernel_3240\909367782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropDuplicatesOrder['order_date'] = pd.to_datetime(dropDuplicatesOrder['order_date'], format='%Y-%m-%d')


In [29]:
dropDuplicatesOrder.iloc[:,-1].unique()[:5]

<DatetimeArray>
['2023-04-09 00:00:00', '2023-04-10 00:00:00', '2023-04-11 00:00:00',
 '2023-04-12 00:00:00', '2023-04-13 00:00:00']
Length: 5, dtype: datetime64[ns]

In [30]:
dropDuplicatesOrder.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2269396 entries, 0 to 2270508
Data columns (total 8 columns):
 #   Column             Dtype         
---  ------             -----         
 0   country_code       object        
 1   order_id           object        
 2   collect_type       object        
 3   payment_method     object        
 4   order_value        float64       
 5   num_items_ordered  float64       
 6   refund_value       float64       
 7   order_date         datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 155.8+ MB


## D. Exporting Cleaned Dataset

In [31]:
order = dropDuplicatesOrder
ogOrderLength = len(ogOrder)
orderLength = len(order)

print(ogOrder.shape)
print(order.shape)
print(f'We removed {ogOrderLength - orderLength} rows which is {((ogOrderLength - orderLength) / ogOrderLength * 100):.2f}% of the total number of rows in the dataset')

(2270509, 8)
(2269396, 8)
We removed 1113 rows which is 0.05% of the total number of rows in the dataset


In [32]:
order.to_csv('final_order.csv', index=False)

# 4. Data Cleansing (Labels)

In [33]:
ogLabels = pd.read_csv("training-labels_v1.0.csv")
ogLabels.head()

,country_code,order_id,customer_id,is_fraud
0,BD,w2lx-myz3,bdpr8uva,0
1,BD,ta7z-r91q,bd59rlzo,0
2,BD,t5af-wgb2,bd6zhjvq,0
3,BD,sibu-9lm4,bd4fv4rb,0
4,BD,we61-omtr,bdzeepq7,0


In [34]:
ogLabels.shape

(2414179, 4)

## A. Checking for Missing Values

In [35]:
ogLabels.isnull().sum()

country_code    0
order_id        0
customer_id     0
is_fraud        0
dtype: int64

## B. Checking for Duplicated Rows

In [36]:
ogLabels.duplicated(subset=['country_code', 'order_id', 'customer_id']).sum()

143612

In [37]:
dropDuplicatesLabels = ogLabels.drop_duplicates(subset=['country_code', 'order_id', 'customer_id'], keep='last')
dropDuplicatesLabels.duplicated().sum()

0

## C. Check Datatypes

In [38]:
dropDuplicatesLabels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2270567 entries, 0 to 2414178
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   country_code  object
 1   order_id      object
 2   customer_id   object
 3   is_fraud      int64 
dtypes: int64(1), object(3)
memory usage: 86.6+ MB


## D. Exporting Cleaned Dataset

In [39]:
labels = dropDuplicatesLabels
ogLabelsLength = len(ogLabels)
labelsLength = len(labels)

print(ogLabels.shape)
print(labels.shape)
print(f'We removed {ogLabelsLength - labelsLength} rows which is {((ogLabelsLength - labelsLength) / ogLabelsLength * 100):.2f}% of the total number of rows in the dataset')

(2414179, 4)
(2270567, 4)
We removed 143612 rows which is 5.95% of the total number of rows in the dataset


In [40]:
labels.to_csv('final_labels.csv', index=False)